# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd

from plotting import *
from prices import *
from imputation import *
from inflation import *
from cost import *

In [3]:
results_dir = 'results/sensitivity_analysis/'
os.makedirs(results_dir, exist_ok=True)

# Cost estimation

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

/Users/bencottier/Projects/training_cost_2/training-cost-trends/data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df.loc[:, 'Training compute (FLOP)'] = pd.to_numeric(


In [5]:
cost_df = estimate_costs(frontier_pcd_df, hardware_df, price_df)
cost_df

==== System: Yuan 1.0 ====
Training time is required but no value found

==== System: OpenAI TI7 DOTA 1v1 ====
Training time is required but no value found

==== System: Claude 2 ====
Training time is required but no value found

==== System: Turing-NLG ====
Training time is required but no value found

==== System: GPT-3.5 (text-davinci-003) ====
Training time is required but no value found

==== System: AlphaGo Master ====
Training time is required but no value found

==== System: Minerva (540B) ====
Trying Google TPU v4 at 2022-04-01 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Found price: 1.45 at 2022-05-11 00:00:00

==== System: FTW ====
Training time is required but no value found

==== System: AlphaGo Zero ====
Trying Google TPU v1 at 2017-07-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price

Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price

Trying Google Cloud, Price per chip-hour (on-demand)
Coul

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Training cost trends,Training cloud compute vendor,Training data center,Archived links,Batch size,Batch size notes,Datasets,Organization categorization,Training time (chip hours),Cost
358,Yuan 1.0,Language,Language modelling,"Shaohua Wu, Xudong Zhao, Tong Yu, Rongguo Zhan...",SOTA Improvement,"""The zero-shot average scores of both LM and P...",NaN,https://arxiv.org/abs/2110.04725,37.0,Yuan 1.0: Large-Scale Pre-trained Language Mod...,...,Yuan 1.0,NaN,NaN,NaN,6881280.0,"Table 2. Batch size 3360, sequence length 2048...",NaN,Industry,NaN,NaN
368,OpenAI TI7 DOTA 1v1,Games,DOTA,NaN,"SOTA Improvement,Historical significance",NaN,NaN,https://openai.com/research/dota-2,0.0,Dota 2,...,OpenAI TI7 DOTA 1v1,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
378,Claude 2,Language,Language modelling,NaN,Historical significance,NaN,API accessible,"https://www.anthropic.com/index/claude-2, http...",0.0,NaN,...,Claude 2,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
428,Turing-NLG,Language,Text autocompletion,Corby Rosset,SOTA Improvement,"from paper: ""Turing Natural Language Generatio...",NaN,https://www.microsoft.com/en-us/research/blog/...,114.0,Turing-NLG: A 17-billion-parameter language mo...,...,Turing-NLG,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
443,GPT-3.5 (text-davinci-003),Language,Language modelling,NaN,"SOTA Improvement,Historical significance,Signi...",NaN,API accessible,https://platform.openai.com/docs/models/gpt-3-5,NaN,NaN,...,GPT-3.5 (text-davinci-003),NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,HyperClova,Language,NaN,"Boseop Kim, HyoungSeok Kim, Sang-Woo Lee, Gich...",SOTA Improvement,"""HyperCLOVA with our training configuration sh...",NaN,https://arxiv.org/abs/2109.04650,79.0,What Changes Can Large-scale Language Models B...,...,HyperClova,NaN,NaN,NaN,NaN,"""All models use the mini-batch size of 1,024""....",NaN,"Industry,Industry",658636.8,948436.992
1274,PanGu-Σ,Language,"Code generation,Language modelling","Xiaozhe Ren, Pingyi Zhou, Xinfan Meng, Xinjing...",SOTA Improvement,"""Our experimental findings show that PanGu-{\S...",NaN,https://arxiv.org/abs/2303.10845,28.0,PanGu-Σ: Towards Trillion Parameter Language M...,...,PanGu-Σ,NaN,NaN,NaN,524288.0,"""We train PanGu-Σ with global batch size of 51...",NaN,Industry,1228800.0,NaN
1276,GLM-130B,Language,NaN,NaN,SOTA Improvement,"""GLM-130B achieves an accuracy of 80.2% on zer...",NaN,https://keg.cs.tsinghua.edu.cn/glm-130b/posts/...,60.0,GLM-130B: An open bilingual pre-trained model,...,GLM-130B,NaN,NaN,NaN,NaN,NaN,NaN,Academia,1105920.0,1603584.000
1277,Flamingo,Multimodal,"Visual question answering,Image captioning","Jean-Baptiste Alayrac, Jeff Donahue, Pauline L...",SOTA Improvement,"""For tasks lying anywhere on this spectrum, a ...",NaN,https://arxiv.org/abs/2204.14198,1320.0,Flamingo: a Visual Language Model for Few-Shot...,...,Flamingo,NaN,NaN,NaN,NaN,NaN,"""MultiModal MassiveWeb, LTIP, VTP, ALIGN""",Industry,552960.0,801792.000


In [6]:
print(cost_df['Cost'].notna().sum(), 'of', len(cost_df), 'systems have a cost estimate')

32 of 68 systems have a cost estimate


In [7]:
print(cost_df['Training time (chip hours)'].notna().sum(), 'of', len(cost_df), 'systems have chip hours')

36 of 68 systems have chip hours


# Sensitivity of cloud prices to fallbacks

In [8]:
# TODO: use the vendor mapping from imputation to reduce repetition
org_to_cloud_vendor = {
    'google': 'Google Cloud',
    'deepmind': 'Google Cloud',
    'microsoft': 'Microsoft Azure',
    'openai': 'Microsoft Azure',
}

pcd_hardware_model_colname = 'Training hardware'
price_colname = 'Price per chip-hour (3-year CUD)'

In [9]:
import contextlib

# Load data
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

# Use the below to track which fallbacks are still active
fallback_names = [
    'price_date_after',
    'default_vendor',
    'backup_vendor',
    'backup_price_type',
]
# Turn off all fallbacks
fallback_is_active = {fallback: False for fallback in fallback_names}
current_fallback_idx = 0
fallback_name = 'price_date_before'
results = pd.DataFrame(columns=['iteration', 'fallback', 'system', 'price'])
iteration = 0
# Loop
while True:
    # Do price matching
    system_to_price = {}

    for i, row in frontier_pcd_df.iterrows():
        # with contextlib.redirect_stdout(None):
        price_value, price_id = find_price(
            row,
            price_df,
            hardware_df,
            pcd_hardware_model_colname,
            price_colname,
            org_to_cloud_vendor,
            **fallback_is_active,
        )
        if price_value is None:
            continue
        else:
            system_to_price[row['System']] = price_value, price_id
            # Store the prices that matched along with their systems
            results.loc[len(results)] = [iteration, fallback_name, row['System'], price_value]
    if all(fallback_is_active.values()):
        break
    # Remove the rows in price_df which have a 'Price source' in system_to_price
    price_df = price_df[~price_df['Price source'].isin([price_id for _, price_id in system_to_price.values()])]
    # If no prices matched, turn on the next fallback
    if len(system_to_price) == 0:
        print(fallback_names[current_fallback_idx], 'is active')
        fallback_name = fallback_names[current_fallback_idx]
        fallback_is_active[fallback_name] = True
        current_fallback_idx += 1
    else:
        iteration += 1

/Users/bencottier/Projects/training_cost_2/training-cost-trends/data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df.loc[:, 'Training compute (FLOP)'] = pd.to_numeric(


==== System: Yuan 1.0 ====
Training time is required but no value found

==== System: OpenAI TI7 DOTA 1v1 ====
Training time is required but no value found

==== System: Claude 2 ====
Training time is required but no value found

==== System: Turing-NLG ====
Training time is required but no value found

==== System: GPT-3.5 (text-davinci-003) ====
Training time is required but no value found

==== System: AlphaGo Master ====
Training time is required but no value found

==== System: Minerva (540B) ====
Trying Google TPU v4 at 2022-04-01 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price

==== System: FTW ====
Training time is required but no value found

==== System: AlphaGo Zero ====
Trying Google TPU v1 at 2017-07-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price

==== System: NASv3 (CIFAR-10) ====
Training time is required but no value found

==== System: AlphaZero ====
Training time is required but no value found

In [10]:
results

,iteration,fallback,system,price
0,0,price_date_before,ALIGN,0.9000
1,0,price_date_before,GPT-4,1.3600
2,0,price_date_before,AlphaStar,0.9000
3,0,price_date_before,GShard (dense),0.9000
4,0,price_date_before,Gopher (280B),0.9000
...,...,...,...,...
58,13,backup_price_type,T5-11B,0.8800
59,13,backup_price_type,BigGAN-deep 512x512,0.8800
60,13,backup_price_type,Switch,0.8800
61,13,backup_price_type,BERT-Large,0.4972


In [11]:
# Find the index at which each fallback was activated
fallback_idxs = results.groupby('fallback').apply(lambda x: x.index[0])
fallback_idxs

fallback
backup_price_type    49
backup_vendor        39
default_vendor       23
price_date_after     19
price_date_before     0
dtype: int64

In [12]:
import plotly.graph_objects as go

fig = go.Figure()

# We want to plot a line of the price over "time" (i.e. the order in which the systems were processed), for each system
for system in results['system'].unique():
    system_results = results[results['system'] == system]
    # Sort by index
    system_results = system_results.sort_index()
    fig.add_trace(go.Scatter(x=system_results.index, y=system_results['price'], mode='lines+markers', name=system))

# Now we want to plot a vertical line at each fallback
for fallback, idx in fallback_idxs.items():
    # Annotate with the fallback name, rotated -90 degrees and aligned to the top of the y axis
    fig.add_annotation(
        x=idx,
        y=4,
        text=fallback,
        showarrow=False,
        textangle=-90,
        xshift=-20,
        xanchor='left',
        yanchor='top',
    )
    fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=idx,
            y0=0,
            x1=idx,
            y1=4,
            # gray dashed line
            line=dict(
                color="Grey",
                width=2,
                dash="dashdot",
            )
        )
    )

# Layout
fig.update_layout(
    title='Effect of fallbacks on cloud price matching',
    xaxis_title='idx',
    yaxis_title='Price matched to system (USD per chip-hour)',
    # legend_title='System',
    showlegend=False,
)

save_plot(fig, results_dir, 'price_fallback_effect_soft_matching')
fig.show()

# Sensitivity of cost estimates to imputation

In [ ]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

In [ ]:
frontier_pcd_df = frontier_pcd_df.set_index('System')
frontier_pcd_df['System'] = frontier_pcd_df.index

In [ ]:
target_col = 'Training hardware'
reference_col = 'Cost'
num_drop = 5
imputer_fn = knn_impute_categorical_column

In [ ]:
known_costs_mask = cost_df['Cost'].notna()
# Filter for rows where all of the required information was known
all_known_df = frontier_pcd_df[known_costs_mask]
all_known_df[target_col]

In [ ]:
# Drop random values where all of the required information was known
dropped_df, holdout_df = drop_random_values(frontier_pcd_df, target_col, cost_df, reference_col, num_drop=num_drop)
dropped_df[known_costs_mask][target_col]

In [ ]:
cost_df_with_imputation = estimate_costs(dropped_df, hardware_df, price_df, impute_pcd_data=True)

In [ ]:
cost_df_with_imputation['Training hardware']

In [ ]:
cost_df_with_imputation['Cost']

In [ ]:
cost_df['Cost']

In [ ]:
# Measure the difference in cost for rows where data was dropped and imputed
holdout_costs_with_imputation = cost_df_with_imputation.loc[holdout_df.index, 'Cost']
holdout_costs = cost_df.loc[holdout_df.index, 'Cost']

holdout_costs_with_imputation

In [ ]:
holdout_costs

In [ ]:
# Calculate the difference
difference = holdout_costs_with_imputation.sub(holdout_costs)

# Calculate percentage difference
percentage_difference = np.abs((difference / cost_df['Cost']) * 100)

# Drop NaN values to keep only Systems with meaningful percentage differences
percentage_difference = percentage_difference.dropna()
percentage_difference

In [ ]:
percentage_difference.mean()

## Scaling up

In [ ]:
def abs_ooms_difference(series1, series2):
    return np.abs(np.log10(series1).sub(np.log10(series2)))

def abs_percentage_difference(series1, series2):
    difference = series1.sub(series2)
    return np.abs((difference / series2) * 100)

In [ ]:
target_col = 'Training hardware'
reference_col = 'Cost'
imputer_fn = knn_impute_categorical_column
difference_metric = abs_ooms_difference
num_drop = 5
num_trials = 1000
print(f'Running {num_trials} trials of {num_drop} dropped values')
print()
differences = []
for i in range(num_trials):
    frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()
    frontier_pcd_df = frontier_pcd_df.set_index('System')
    frontier_pcd_df['System'] = frontier_pcd_df.index

    # Drop random values where all of the required information was known
    dropped_df, holdout_df = drop_random_values(frontier_pcd_df, target_col, cost_df, reference_col, num_drop=num_drop)

    # Estimate costs with imputation
    cost_df_with_imputation = estimate_costs(dropped_df, hardware_df, price_df, impute_pcd_data=True)

    # Measure the difference in cost for rows where data was dropped and imputed
    holdout_costs_with_imputation = cost_df_with_imputation.loc[holdout_df.index, reference_col]
    holdout_costs = cost_df.loc[holdout_df.index, reference_col]

    # Calculate the difference
    difference = difference_metric(holdout_costs_with_imputation, holdout_costs)
    differences.extend(difference.dropna().to_list())
print()
print(f'Mean {difference_metric.__name__}: {np.mean(differences)}')
print(f'Std {difference_metric.__name__}: {np.std(differences)}')
print(f'2.5%, 50%, 97.5%: {np.percentile(differences, [2.5, 50, 97.5])}')